# Pollen Data Visualisation
The purpose of this notebook is to:
- Load fossilised pollen assemblage and chronology data from the European Pollen Database (EPD)
- Build interactive visualisations to explore the data
- Derive hypotheses as to the causal drivers of trends observed in the data

In [1]:
from sqlalchemy import create_engine
import pandas as pd
import os

Prior work has idendtified six sites as being of particular interest due to their temporal coverage of the majority of the Holocene, their spatial distribution around Iberia, and the fact that many of them provide evidence of human involvement in changing the landscape. 

These sites all have records stored in the EPD, and can be uniquely identified using the `sitecode` field in the `siteloc` table. These site names and corresponding sitecodes are stored in the dictionary `ssites' 

In [2]:
ssites = {'San Rafael' : 'ESP-01000-SANR',
          'Albufera de Alcudia' : 'ESP-04000-ALBU',
          'Navarres' : 'ESP-11000-NAVA',
          'Laguna Guallar' : 'ESP-02000-LAGU',
          'Monte Areo mire' : 'ESP-03000-AREO',
          'Sanabria Marsh' : 'ESP-08000-SANA'}

Create a database engine for the session

In [3]:
engine = create_engine('postgresql://andrew@localhost:5432/epd95')

### Get site location information

Function to return the site location information for a list of site codes

In [4]:
def get_site_loc_info(site_code_list):
    #construct string for postgreql WHERE condition based on input array
    where_condition = ' OR '.join(["sitecode = '"+x+"'" for x in ssites.values()])
    query = "SELECT * FROM siteloc WHERE " + where_condition
    
    # return results in pandas dataframe
    return pd.read_sql_query(query, con=engine)   

In [5]:
site_loc_info = get_site_loc_info(ssites.values())
site_loc_info.set_index('site_',inplace=True)
print site_loc_info

                     sitename        sitecode siteexists poldiv1 poldiv2  \
site_                                                                      
44             Sanabria Marsh  ESP-08000-SANA       None     ESP      08   
396                  Navarrés  ESP-11000-NAVA       None     ESP      11   
486                San Rafael  ESP-01000-SANR       None     ESP      01   
759          Albufera Alcudia  ESP-04000-ALBU       None     ESP      04   
761            Laguna Guallar  ESP-02000-LAGU       None     ESP      02   
784    Laguna Salada Chiprana  ESP-02000-LAGU       None     ESP      02   
1252          Monte Areo mire  ESP-03000-AREO       None     ESP      03   

      poldiv3  latdeg  latmin  latsec latns      latdd     latdms  londeg  \
site_                                                                       
44        000      42       6       0     N  42.100000  42.06.00N       6   
396       000      39       6       0     N  39.100000  39.06.00N       0   
486    

Note that site no. 784  Laguna Salada Chiprana seems to have same sitecode as site no. as our study site no. 761  Laguna Guallar. Let's drop this site to avoid confusion later on.

In [6]:
site_loc_info.drop(784,inplace=True)
print site_loc_info

               sitename        sitecode siteexists poldiv1 poldiv2 poldiv3  \
site_                                                                        
44       Sanabria Marsh  ESP-08000-SANA       None     ESP      08     000   
396            Navarrés  ESP-11000-NAVA       None     ESP      11     000   
486          San Rafael  ESP-01000-SANR       None     ESP      01     000   
759    Albufera Alcudia  ESP-04000-ALBU       None     ESP      04     000   
761      Laguna Guallar  ESP-02000-LAGU       None     ESP      02     000   
1252    Monte Areo mire  ESP-03000-AREO       None     ESP      03     000   

       latdeg  latmin  latsec latns      latdd     latdms  londeg  lonmin  \
site_                                                                       
44         42       6       0     N  42.100000  42.06.00N       6      44   
396        39       6       0     N  39.100000  39.06.00N       0      41   
486        36      46      25     N  36.773611  36.46.25N       2  

### Identify sediment cores associated with each site

The EPD [documentation](http://www.europeanpollendatabase.net/data/downloads/image/pollen-database-manual-20071011.doc) explains that there are potentially multiple 'entities' associated with each site location. An 'entity' simply refers to a sediment core, section or surface sample.

Define a function to return all entities (or cores) associated with a given site number:

In [7]:
def get_entity_info(site_number):
    # query finds all fields from entity table for records 
    # matching given site_number.
    query = "SELECT * FROM entity WHERE site_ = {0}".format(site_number)
             
    # return results in pandas dataframe
    return pd.read_sql_query(query, con=engine)   

Create a dataframe containing entity information for all study sites

In [8]:
ssite_numbers = site_loc_info.index
for i in range(len(ssite_numbers)):
    if i == 0:
        site_entity_info = get_entity_info(ssite_numbers[i])
    else:
        site_entity_info = site_entity_info.append(get_entity_info(ssite_numbers[i]), ignore_index=True)
site_entity_info.set_index('e_',inplace=True)

In [9]:
site_entity_info = site_entity_info.join(site_loc_info, on='site_', how='left')

site_entity_info = site_entity_info.drop(['latns','latdd','latdms','londeg','lonmin',u'poldiv1',
                      u'poldiv2',u'poldiv3',u'latdeg',u'latmin',u'latsec',
                      u'lonsec', u'lonew', u'londd', u'londms', u'elevation', u'areaofsite',
                      u'depthatloc', u'icethickcm', u'sampdevice', u'corediamcm',
                      u'c14depthadj', u'notes','siteexists','localveg','hasanlam','coll_'],axis=1)

In [10]:
print site_entity_info 

      site_     sigle    name iscore issect isssamp descriptor  \
e_                                                               
44       44  SANABRIA    None      Y      N       Y       PMSH   
469     396     NAVA1  core 1      Y      N       Y       PMIR   
470     396     NAVA2  core 2      Y      N       Y       PMIR   
471     396  NAVARRE3    None      Y      N       Y       UNKN   
574     486   SANRAFA    None      Y      N       Y       MCOA   
891     759   ALCUDIA    None      Y      N       Y       MCOA   
893     761     N-GUA  core 1      Y      N       Y       LNPL   
1562   1252      AREO    None      Y      N       N       PMIR   

                     entloc    sampdate          sitename        sitecode  
e_                                                                         
44                     None  1981-05-00    Sanabria Marsh  ESP-08000-SANA  
469        Center of valley  1993-06-15          Navarrés  ESP-11000-NAVA  
470        Center of valley  1993-0

Noting the multiple entities for Navares (site 396), I searched the EPD for publications relating to Navares entities:

In [66]:
pd.read_sql_query("""SELECT publent.e_, publent.publ_,publ.citation
                     FROM publ 
                     INNER JOIN publent
                     ON publ.publ_=publent.publ_
                     WHERE e_=469 OR e_=470 OR e_=471
                     ORDER BY publent.e_;""", con=engine)

,e_,publ_,citation
0,469,134,"Carrión, J.S., and M. Dupré-Olivier. 1996. Lat..."
1,470,134,"Carrión, J.S., and M. Dupré-Olivier. 1996. Lat..."
2,471,133,"Carrión, J.S., and B. van Geel. 1999. Fine-res..."


We find entities 469 and 470 come from a [1996 paper](http://onlinelibrary.wiley.com/doi/10.1111/j.1469-8137.1996.tb01157.x/abstract;jsessionid=5DEA525DEED061D4D29979D7C2AEFCCD.f02t02) in which the authors discuss both:
- A comparison of the palynological results produced by two separate cores extracted 5m apart from each other. 
- The role of humans in causing an observed switch from /Pinus/ to /Quercus/. 
These cores cover the temporal range 20700 to 3075 yr BP.

The entity 471, NAVARRE3, relates to a [1999 paper](https://www.researchgate.net/publication/222507129_Fine-resolution_Upper_Weichselian_and_Holocene_palynological_record_from_Navarres_Valencia_Spain_and_a_discussion_about_factors_of_Mediterranean_forest_succession) featuring a finer temporal resolution analysis and -- according to the paper -- charcoal analysis. However, this charcoal data does not seem to have been made available (either in the EPD or Neotoma, using nav <- get_site('Navarrés'), dat <- get_dataset(nav)) and the temporal range covered is 30900 to 3160 yr BP. 

The possibility of comparing results between cores at the same site and the more detailed discussion about human drivers in the 1996 paper have led me to focus on the entities 469 and 470 (NAVA1 and NAVA2 respectively) for the purpose of this analysis. 

I will therefore drop entity 471 (NAVARRE3) from the `site_entity_info` dataframe. 

In [67]:
site_entity_info = site_entity_info.drop([471])

In [68]:
print site_entity_info

      site_     sigle    name iscore issect isssamp descriptor  \
e_                                                               
44       44  SANABRIA    None      Y      N       Y       PMSH   
469     396     NAVA1  core 1      Y      N       Y       PMIR   
470     396     NAVA2  core 2      Y      N       Y       PMIR   
574     486   SANRAFA    None      Y      N       Y       MCOA   
891     759   ALCUDIA    None      Y      N       Y       MCOA   
893     761     N-GUA  core 1      Y      N       Y       LNPL   
1562   1252      AREO    None      Y      N       N       PMIR   

                     entloc    sampdate          sitename        sitecode  
e_                                                                         
44                     None  1981-05-00    Sanabria Marsh  ESP-08000-SANA  
469        Center of valley  1993-06-15          Navarrés  ESP-11000-NAVA  
470        Center of valley  1993-06-15          Navarrés  ESP-11000-NAVA  
574         East of Almer

### Get pollen counts

Function which extracts pollen count data for a given entity (i.e. sediment core) number and joins it to the p_vars table to incorporate species data useful to subsequent analysis.

In [69]:
def get_p_count(entity):
    query="""SELECT p_counts.e_,p_counts.sample_,p_counts.var_,
             p_counts.count,p_vars.varcode,p_vars.varname 
             FROM p_counts
             LEFT JOIN p_vars
             ON p_counts.var_=p_vars.var_
             WHERE p_counts.e_={0};""".format(entity)
    return pd.read_sql_query(query, con=engine)   

Example of output for the San Rafael entitity (number 574)

In [70]:
get_p_count(574).head()

,e_,sample_,var_,count,varcode,varname
0,574,1,66,27.0,Art,Artemisia
1,574,1,95,1.0,Bet,Betula
2,574,1,165,2.0,Crl-T,Cerealia-type
3,574,1,185,108.0,Cheae,Chenopodiaceae
4,574,1,196,1.0,Ciu,Cistus


Write pollen count data to file for each of the entities identified in the `site_entity_info` dataframe.

In [71]:
for e in site_entity_info.index:
    e_dat = site_entity_info.loc[e]
    # construct file name string using data from site_entity_info dataframe
    s = 'e'+str(e)+'_s'+str(e_dat.site_)+'_'+e_dat.sitename.replace(' ','_').replace(u'é','e')+\
        '_pcounts.csv'
        
    p_dat = get_p_count(e) # dataframe containing pollen count data for entity
    p_dat.drop('e_',axis=1).to_csv(os.path.join('data',s),index=False)

To avoid needing to access the database to work with this data in future, we define a function to load pollen count data from the `.csv` files produced in the previous step.

In [72]:
def get_p_count_from_file(entity):
    file_candidates=[]
    all_files=os.listdir('data') #list of all files in data directory
    for f_name in all_files:
        name_components=f_name.split('_') #split on underscores in file names
        if (name_components[0]=='e'+str(entity) #check files for entity number
            and name_components[-1]=='pcounts.csv'): #check corresponds to pollen counts
            file_candidates.append(f_name) #add to list of possible files
            
    if len(file_candidates) == 0:
        print """No files found. Check pollen data file exists and entity number correct."""
    elif len(file_candidates) > 1:
        print """Pollen data file not uniquely identified using entity number. Check data files"""
    else:
        try:
            dat=pd.read_csv(os.path.join('data',file_candidates[0]))
            print "pollen file for entity {0} read.".format(entity)
            dat['_e']=entity # reinstate entity number column (removed in file write)
            return dat
        
        except IOError as e:
            print """Unique pollen file for entity {0} found but couldn't be read\n
            IO Error: {1}""".format(entity, e)

In [73]:
get_p_count_from_file(574).head()

pollen file for entity 574 read.


,sample_,var_,count,varcode,varname,_e
0,1,66,27.0,Art,Artemisia,574
1,1,95,1.0,Bet,Betula,574
2,1,165,2.0,Crl-T,Cerealia-type,574
3,1,185,108.0,Cheae,Chenopodiaceae,574
4,1,196,1.0,Ciu,Cistus,574


### Get chronology for samples

Chronologies apply to /samples/ within entities (cores). These chronologies are the results of models which are used to interpolate the age of all samples in a given core based on C14 testing of a subset of those samples.

For some cores there are multiple chronologies supplied in the EPD. In these instances, the chronology which is marked as **default** in the database is the one which is extracted (see [documentation](http://www.europeanpollendatabase.net/data/downloads/image/pollen-database-manual-20071011.doc) for details).

Information about chronologies for each core is provided in the `chron` table. The inferred fitted chronology for pollen samples is contained in the `p_adedpt` table.

Additionally, we can find the dates corresponding to the samples which were actually tested in the `c14` table, but this will not enter into our analyses here.

We now define a function for extracting the fitted chronology for a given entity:

In [74]:
def get_chronology(entity):
    # get id of database default chronology for selected entity
    def_chron_no=pd.read_sql_query("SELECT chron_ FROM chron WHERE e_={0} AND defaultchron='Y'".format(entity), 
                                   con=engine)['chron_'].values[0]  
    
    query = "SELECT * FROM p_agedpt WHERE e_={0} AND chron_={1}".format(entity,def_chron_no)
    return pd.read_sql_query(query,con=engine).drop(['chron_'],axis=1)


Sample output for Laguna Guallar (entity 893)

In [75]:
get_chronology(893).head()

,e_,sample_,agebp,ageup,agelo,deptime
0,893,1,0.0,None,None,None
1,893,2,982.0,None,None,None
2,893,3,1963.0,None,None,None
3,893,4,2945.0,None,None,None
4,893,5,3927.0,None,None,None


Write chronologies for each entity to data folder

In [76]:
for e in site_entity_info.index:
    e_dat = site_entity_info.loc[e]
    # construct file name string using data from site_entity_info dataframe
    s = 'e'+str(e)+'_s'+str(e_dat.site_)+'_'+e_dat.sitename.replace(' ','_').replace(u'é','e')+\
        '_chron.csv'
        
    p_dat = get_chronology(e) # dataframe containing chronology data for entity
    p_dat.drop('e_',axis=1).to_csv(os.path.join('data',s),index=False)

To avoid needing to access the database to work with this data in future, we define a function to load chronology data from the .csv files produced in the previous step.

In [77]:
def get_chronology_from_file(entity):
    file_candidates=[]
    all_files=os.listdir('data') #list of all files in data directory
    for f_name in all_files:
        name_components=f_name.split('_') #split on underscores in file names
        if (name_components[0]=='e'+str(entity) #check files for entity number
            and name_components[-1]=='chron.csv'): #check corresponds to pollen counts
            file_candidates.append(f_name) #add to list of possible files
            
    if len(file_candidates) == 0:
        print """No files found. Check chronology data file exists and entity number correct."""
    elif len(file_candidates) > 1:
        print """Chronology data file not uniquely identified using entity number. Check data files"""
    else:
        try:
            dat=pd.read_csv(os.path.join('data',file_candidates[0]))
            print "Chronology file for entity {0} read.".format(entity)
            dat['_e']=entity # reinstate entity number column (removed in file write)
            return dat
        
        except IOError as e:
            print """Unique chronology file for entity {0} found but couldn't be read\n
            IO Error: {1}""".format(entity, e)

Test loading chronology data for entity 893 from file:

In [78]:
get_chronology_from_file(893).head()

Chronology file for entity 893 read.


,sample_,agebp,ageup,agelo,deptime,_e
0,1,0.0,NaN,NaN,NaN,893
1,2,982.0,NaN,NaN,NaN,893
2,3,1963.0,NaN,NaN,NaN,893
3,4,2945.0,NaN,NaN,NaN,893
4,5,3927.0,NaN,NaN,NaN,893


### Assess which taxa should be plotted in visualisations
Consider the pollen count data from entity 44, the sediment core from Sanabria Marsh:

In [79]:
SAN_p = get_p_count_from_file(44)
print SAN_p.head()

pollen file for entity 44 read.
   sample_  var_  count varcode           varname  _e
0        1     1    1.0     Abi             Abies  44
1        1    32    7.0     Aln             Alnus  44
2        1    66    3.0     Art         Artemisia  44
3        1    95    4.0     Bet            Betula  44
4        1   124   10.0   Cal.v  Calluna vulgaris  44


The number of species identified from their pollen in this core is given by

In [80]:
"The number of species identified from their pollen in this core is {0}.".format(len(SAN_p.groupby('varname').count().index))

'The number of species identified from their pollen in this core is 94.'

Clearly this is too many for us to be able to plot them all coherently on a single graph. Therefore we will need to _aggregate_ the data by grouping species together in a principled way, and/ or by choosing not to plot some species altogether.

First, we obtain a list of all taxa which have been identified in the cores we are considering as listed in the `site_entity_info` table. 

In [82]:
site_entity_info.index

Int64Index([44, 469, 470, 574, 891, 893, 1562], dtype='int64', name=u'e_')

In [87]:
entity_numbers = site_entity_info.index
# go through all entities, load pollen data from file
# and append into one dataframe called all_pollen_data
for i in range(len(entity_numbers)):
    if i == 0:
        all_pollen_data = get_p_count_from_file(entity_numbers[i])
    else:
        all_pollen_data = all_pollen_data.append(get_p_count_from_file(entity_numbers[i]), ignore_index=True)

pollen file for entity 44 read.
pollen file for entity 469 read.
pollen file for entity 470 read.
pollen file for entity 574 read.
pollen file for entity 891 read.
pollen file for entity 893 read.
pollen file for entity 1562 read.


In [89]:
print all_pollen_data.head()
print ""
print all_pollen_data.tail()

   sample_  var_  count varcode           varname  _e
0        1     1    1.0     Abi             Abies  44
1        1    32    7.0     Aln             Alnus  44
2        1    66    3.0     Art         Artemisia  44
3        1    95    4.0     Bet            Betula  44
4        1   124   10.0   Cal.v  Calluna vulgaris  44

       sample_  var_  count   varcode                      varname    _e
10119       55  3028   68.0     Eri-T                   Erica-type  1562
10120       55  3462   25.0      T114                     Type 114  1562
10121       55  3911   23.0  Flc.m.ud  Filicales monoletes undiff.  1562
10122       55  4274    3.0  Spi.cf.s   Spirogyra cf. scrobiculata  1562
10123       55  4366   81.0  Que.py-T       Quercus pyrenaica-type  1562


Group by varcode and varname to obtain a list of unique values

In [100]:
all_codes_names = all_pollen_data.groupby(['varcode','varname']).count().reset_index()[['varcode','varname']]
all_codes_names=all_codes_names.sort_values('varname')
print all_codes_names.head()
print ""
print "A total of {0} species identified across all 7 cores considered.".format(len(all_codes_names.index))

  varcode            varname
1     Abi              Abies
2     Ace               Acer
3     Alc         Alchemilla
4     Aln              Alnus
5   Amp.f  Amphitrema flavum

A total of 244 species identified across all 7 cores considered.


Write this list to a .csv file. This file can then be opened in a spreadsheet to be easily annotated. We will then read this back into this notebook.

In [101]:
all_codes_names.to_csv('all_species_unannotated.csv',index=False)

In [103]:
species_table = pd.read_csv('all_species_annotated.csv')
print species_table.dropna()

      varcode                   varname include  \
3         Aln                     Alnus       Y   
30        Cas                  Castanea       Y   
35        Crl                  Cerealia       Y   
36      Crl-T             Cerealia-type       Y   
69      Eriae                 Ericaceae       Y   
76        Fag                     Fagus       Y   
83        Frx                  Fraxinus       Y   
146       Pin                     Pinus       Y   
147     Pis.p            Pinus pinaster       Y   
148   Pin.s-T     Pinus sylvestris-type       Y   
150       Pla                  Plantago       Y   
151   Pla.c-T   Plantago coronopus-type       Y   
152   Pla.l-T  Plantago lanceolata-type       Y   
153   Pla.m/m   Plantago major/P. media       Y   
154    Pla.ud          Plantago undiff.       Y   
170       Que                   Quercus       Y   
171     Que.i              Quercus ilex       Y   
172   Que.i-T         Quercus ilex-type       Y   
173  Que.py-T    Quercus pyrena